### 1. Check brackets

In [ ]:
from collections import namedtuple
import re 
Bracket = namedtuple("Bracket", ["char", "position"])

input_str = '(())'
def find_mismatch(input_str: str) -> None | int:
    stack = []
    for index, char in enumerate(input_str):
        if char in ['{', '[', '(']:
            stack.append(char)
        elif char in ['}', ']', ')']:
            latest_open_bracket = stack.pop()
            if latest_open_bracket == '{':
                if char != '}':
                    return index+1
            elif latest_open_bracket == '[':
                if char != ']':
                    return index+1
            elif latest_open_bracket == '(':
                if char != ')':
                    return index+1
        else:
            continue
    
    if len(stack) == 0:
        return "Success"
    return len(input_str)
                
find_mismatch(input_str)

### 2. Compute tree height

In [ ]:
# def compute_height(n, parents):
#     ...

from dataclasses import dataclass, field
from typing import Type

@dataclass
class Node:
    index: int
    parent: list[Type['Node']] = field(default_factory=list)
    child: list[Type['Node']] = field(default_factory=list)

input = [-1, 0, 4, 0, 3]

## build tree: O(N)
def build_tree(input):
    all_nodes = [Node(index=index) for index in range(len(input))]
    for index, node in enumerate(all_nodes):
        if input[index] == -1:
            root = node
            node.parent = None
        else:
            node.parent = all_nodes[input[index]]
            node.parent.child.append(all_nodes[index])
    
    return root
    
root = build_tree(input)

## Find height: O(N)
def find_height(root):
    if root.child == []:
        return 1
    max_height = 0
    for child in root.child:
        subtree_root = child
        subtree_height = find_height(subtree_root)
        max_height = max(
            max_height, 
            subtree_height+1
        )
    return max_height

# find_height(root)

In [55]:
class Node:
    def __init__(self, index, parent=[], child=[]):
        self.index = index
        self.parent = parent
        self.child = child
    
    def __repr__(self):
        return f"Node({self.index=}, {self.parent=}, {self.child=})"

def build_tree(parents):
    all_nodes = [Node(index=index, parent=[], child=[]) for index in range(len(parents))]

    for index in range(len(all_nodes)):        
        if parents[index] == -1:
            root = all_nodes[index]
            all_nodes[index].parent = None
        else:
            all_nodes[index].parent = all_nodes[parents[index]]
            all_nodes[index].parent.child.append(all_nodes[index])
                
    return root, all_nodes

def compute_height(root):    
    if root.child == []:
        return 1
    max_height = 0
    for child in root.child:
        subtree_root = child
        subtree_height = compute_height(subtree_root)
        max_height = max(
            max_height, 
            subtree_height+1
        )
    return max_height

parents = [-1,0,4,0,3]
# root, all_nodes = build_tree(parents)
root, all_nodes = build_tree(parents)
print(compute_height(root))

4


| | with open lineage | without open lineage |
| - | - | - |
| log level debug | succeed | succeed |
| log level warn | fail | ? |

### 3. Network packet processing simulation

In [81]:
from collections import namedtuple

Request = namedtuple("Request", ["arrived_at", "time_to_process"])
Response = namedtuple("Response", ["was_dropped", "started_at"])

In [90]:
from collections import deque
buffer_size, n_requests = 1, 0
all_requests = None #[(0,0)]

class Buffer:
    def __init__(self, size):
        self.size = size
        self.finish_time = deque()

    def _process(self, request):
        time_start_processing, time_end_processing = request

        ## if nothing in queue...
        if not self.finish_time:
            ## add the end of the processing time to finish time
            self.finish_time.append(time_end_processing)
            return Response(False, time_start_processing)
            
        ## else if something in queue...
        else:
            queue_len = len(self.finish_time)
            if time_start_processing >= self.finish_time[-1]:
                self.finish_time.popleft()
                self.finish_time.append(time_end_processing)
                return Response(False, time_start_processing)

            ## if queue length is max, check if we can process this now
            if queue_len >= self.size:
                return Response(True, -1)
            else:
                return Response(False, time_start_processing)

    def process_requests(self, requests):
        responses = []
        for request in requests:
            response = self._process(request)
            responses.append(response)
        return responses

requests = []
for i in range(n_requests):    
    arrived_at, time_to_process = all_requests[i]
    requests.append(Request(arrived_at, time_to_process))

buffer = Buffer(buffer_size)
responses = buffer.process_requests(requests)

for response in responses:
    print(response.started_at if not response.was_dropped else -1)

### 4. Extending stack interface

In [ ]:
# Class to make a Node 
class Node:
    #Constructor which assign argument to nade's value 
    def __init__(self, value):
        self.value = value
        self.next = None
   
    # This method returns the string representation of the object.
    def __str__(self):
        return f"Node({self.value})"
     
    #  __repr__ is same as __str__
    __repr__ = __str__   
 
class Stack:
    # Stack Constructor initialise top of stack and counter.
    def __init__(self):
        self.top = None
        self.count = 0
        self.maximum = None
         
    #This method returns the string representation of the object (stack).
    def __str__(self):
        temp=self.top
        out=[]
        while temp:
            out.append(str(temp.value))
            temp=temp.next
        out='\n'.join(out)
        return ('Top {} \n\nStack :\n{}'.format(self.top,out))
         
    #  __repr__ is same as __str__
    __repr__=__str__
     
    #This method is used to get minimum element of stack
    def getMax(self):
        if self.top is None:
            return "Stack is empty"
        else:
            print("Maximum Element in the stack is: {}" .format(self.maximum))
 
  
 
    # Method to check if Stack is Empty or not
    def isEmpty(self):
        # If top equals to None then stack is empty 
        if self.top == None:
            return True
        else:
        # If top not equal to None then stack is empty 
            return False
 
    # This method returns length of stack       
    def __len__(self):
        self.count = 0
        tempNode = self.top
        while tempNode:
            tempNode = tempNode.next
            self.count+=1
        return self.count
 
    # This method returns top of stack       
    def peek(self):
        if self.top is None:
            print ("Stack is empty")
        else:    
            if self.top.value > self.maximum:
                print("Top Most Element is: {}" .format(self.maximum))
            else:
                print("Top Most Element is: {}" .format(self.top.value))
 
    #This method is used to add node to stack
    def push(self,value):
        if self.top is None:
            self.top = Node(value)
            self.maximum = value
             
        elif value > self.maximum :
            temp = (2 * value) - self.maximum
            new_node = Node(temp)
            new_node.next = self.top
            self.top = new_node
            self.maximum = value
        else:
            new_node = Node(value)
            new_node.next = self.top
            self.top = new_node
        print("Number Inserted: {}" .format(value))
   
    #This method is used to pop top of stack
    def pop(self):
        if self.top is None:
            print( "Stack is empty")
        else:
            removedNode = self.top.value
            self.top = self.top.next
            if removedNode > self.maximum:
                print ("Top Most Element Removed :{} " .format(self.maximum))
                self.maximum = ( ( 2 * self.maximum ) - removedNode )
            else:
                print ("Top Most Element Removed : {}" .format(removedNode))
 
                 
             
     
 # Driver program to test above class  
stack = Stack() 
 
stack.push(3)
stack.push(5) 
stack.getMax()
stack.push(7)
stack.push(19)
stack.getMax()     
stack.pop()
stack.getMax()
stack.pop() 
stack.peek()
 
# This code is contributed by Blinkii